<a href="https://colab.research.google.com/github/MYTHILIPRIYA96/Zomato-Data-Insights/blob/main/Zomato_Data_Insights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install libraries
!pip install mysql-connector-python
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.6 MB/s eta 0:00:00


In [ ]:
#Import Required Libraries
import mysql.connector
from faker import Faker
fake = Faker(["en_IN"])
import random
from datetime import datetime, timedelta#for working with dates and times.

In [ ]:
# Define Database class
class Database:#defines a class called Database
#defines a blueprint for creating objects
    def __init__(self, host, user, password, port, database):#This is the constructor
        self.host = host
        self.user = user
        self.password = password
        self.port = port
        self.database = database
        self.connection = None
        self.cursor = None


    def connect(self):
        self.connection = mysql.connector.connect(
            host=self.host,
            user=self.user,
            password=self.password,
            port=self.port
        )
        self.cursor = self.connection.cursor()# creates a cursor object
        print("connection successfully")
        self.create_database()
        self.cursor.execute(f"USE {self.database}")
        self.connection.commit()

    def create_database(self):

        if self.connection is None:
            self.connect()

        sql = "CREATE DATABASE IF NOT EXISTS Zomato"
        self.cursor.execute(sql)
        self.connection.commit()
        print("Database successfully created")

    def create_customers_table(self):
        sql = """
        CREATE TABLE IF NOT EXISTS customers (
            customer_id VARCHAR(36) PRIMARY KEY,
            name VARCHAR(30),
            email VARCHAR(50),
            phone VARCHAR(20),
            location TEXT,
            signup_date DATETIME,
            is_premium BOOLEAN,
            preferred_cuisine VARCHAR(20),
            total_orders INT,
            average_rating decimal(2, 1)
        )
        """
        self.cursor.execute(sql)
        self.connection.commit()

    def insert_customers(self, data):
        sql = """INSERT INTO
               customers(customer_id,name,email,phone,location,signup_date,is_premium,preferred_cuisine,total_orders,average_rating)
               VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
        self.cursor.executemany(sql, data)
        self.connection.commit()

    def create_restaurants_table(self):
        sql = """
        CREATE TABLE IF NOT EXISTS restaurants
        (
            restaurant_id VARCHAR(36) PRIMARY KEY,
            name VARCHAR(30),
            cuisine_type VARCHAR(20),
            location VARCHAR(200),
            owner_name VARCHAR(30),
            average_delivery_time INT,
            contact_number VARCHAR(20),
            rating DECIMAL(3,2),
            total_orders INT,
            is_active BOOLEAN
        )
        """
        self.cursor.execute(sql)
        self.connection.commit()

    def insert_restaurants(self, data):
        sql = """INSERT INTO
                  restaurants(restaurant_id,name,cuisine_type,owner_name,location,average_delivery_time,contact_number,rating,total_orders,is_active)
                  VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
              """
        self.cursor.executemany(sql, data)
        self.connection.commit()



    def create_orders_table(self):
        sql = """
              create table IF NOT EXISTS orders
              (
              order_id VARCHAR(36) PRIMARY KEY,
              customer_id VARCHAR(36),
              restaurant_id VARCHAR(36),
              order_date DATETIME,
              delivery_time DATETIME,
              status VARCHAR(25),
              total_amount DECIMAL(12,2),
              payment_mode VARCHAR(25),
              discount_applied DECIMAL(10,2),
              feedback_rating VARCHAR(25),
              FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
              FOREIGN KEY (restaurant_id) REFERENCES restaurants(restaurant_id)
              )
              """
        self.cursor.execute(sql)
        self.connection.commit()

    def insert_orders(self, data):
        sql = """INSERT INTO orders(order_id,customer_id,restaurant_id,order_date,delivery_time,status,total_amount,payment_mode,discount_applied,feedback_rating)
                VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
        self.cursor.executemany(sql, data)
        self.connection.commit()



    def create_delivery_persons_table(self):
       sql ="""
        CREATE TABLE IF NOT EXISTS delivery_persons (
            delivery_person_id VARCHAR(36) PRIMARY KEY,
            name VARCHAR(50),
            contact_number VARCHAR(25),
            vehicle_type VARCHAR(25),
            total_deliveries INT,
            average_rating DECIMAL(5,2),
            location VARCHAR(200)
        )
        """
       self.cursor.execute(sql)
       self.connection.commit()

    def insert_delivery_persons(self,data):
        sql = """INSERT INTO delivery_persons(delivery_person_id,name,contact_number,vehicle_type,total_deliveries,average_rating,location)
               VALUES (%s,%s,%s,%s,%s,%s,%s)"""
        self.cursor.executemany(sql,data)
        self.connection.commit()

    def create_deliveries_table(self):
        sql = """
            create table if not exists deliveries
            (
            delivery_id VARCHAR(36) PRIMARY KEY,
            order_id VARCHAR(36),
            delivery_person_id VARCHAR(36),
            delivery_status VARCHAR(36),
            distance DECIMAL(5,2),
            estimated_time INT,
            delivered_time DATETIME,
            delivery_fee DECIMAL(5,2),
            vehicle_type VARCHAR(25),
            FOREIGN KEY (order_id) REFERENCES orders(order_id),
            FOREIGN KEY (delivery_person_id) REFERENCES delivery_persons(delivery_person_id)
            )
            """
        self.cursor.execute(sql)
        self.connection.commit()

    def insert_deliveries(self, data):
        sql = """INSERT INTO deliveries(delivery_id,order_id,delivery_person_id,delivery_status,distance,estimated_time,delivered_time,delivery_fee,vehicle_type)
               VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
        self.cursor.executemany(sql, data)
        self.connection.commit()

# Create a Database instance and perform operations
db = Database(
    host="gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
    user="3obbBvALArqQPqW.root",
    password="MtV1PAc27naJolYM",
    port="4000",
    database="Zomato",
)

db.create_database()


# Create and populate tables
db.create_customers_table()
customers_data = [
    [
        fake.uuid4(),
        fake.name(),
        fake.unique.email(),
        fake.phone_number(),
        fake.address(),
        fake.date_time_between(start_date="-2y", end_date="now"),
        random.choice([True, False]),
        random.choice(["Indian", "Chinese", "Mexican", "Italian", "Japanese", "French"]),
        random.randint(1,5000),
        round(random.uniform(1.0, 5.0), 1)
    ]
    for _ in range(500)
]
db.insert_customers(customers_data)

db.create_restaurants_table()
restaurants_data = [
    [
        fake.uuid4(),
        fake.name(),
        random.choice(["Indian", "Chinese", "Mexican", "Italian", "Japanese", "French"]),
        fake.name(),
        fake.address(),
        random.randint(20, 60),
        fake.phone_number(),
        round(random.uniform(1, 5), 1),
        random.randint(50, 5000),
        random.choice([True, False]),
    ]
    for _ in range(500)
]
db.insert_restaurants(restaurants_data)



db.create_orders_table()
db.cursor.execute("SELECT customer_id FROM customers")
customer_ids = [row[0] for row in db.cursor.fetchall()]
db.cursor.execute("SELECT restaurant_id FROM restaurants")
restaurant_ids = [row[0] for row in db.cursor.fetchall()]

orders_data = []

for _ in range(500):
    orders_date = fake.date_time_between(start_date="-2y",end_date="now")
    delivery_time = orders_date + timedelta(minutes=random.randint(20,60))

    orders_data.append(
    [
        fake.uuid4(),
        random.choice(customer_ids),
        random.choice(restaurant_ids),
        orders_date.strftime("%Y-%m-%d %H:%M:%S"),
        delivery_time.strftime("%Y-%m-%d %H:%M:%S"),
        random.choice(["Pending", "Delivered", "Cancelled"]),
        round(random.randint(100, 5000),2),
        random.choice(["credit card", "cash", "UPI", "debit card", "net banking"]),
        round(random.uniform(0, 500), 2),
        round(random.uniform(1, 5), 1)
    ])


db.insert_orders(orders_data)

db.create_delivery_persons_table()
delivery_persons_data = [
    [
        fake.uuid4(),
        fake.name(),
        fake.phone_number(),
        random.choice(["Bike", "Car", "Bicycle"]),
        random.randint(100, 2000),
        round(random.uniform(1, 5), 1),
        fake.city(),
    ]
    for _ in range(500)
]
db.insert_delivery_persons(delivery_persons_data)

db.create_deliveries_table()
db.cursor.execute("SELECT order_id FROM orders")
order_ids = [row[0] for row in db.cursor.fetchall()]
db.cursor.execute("SELECT delivery_person_id FROM delivery_persons")
delivery_person_ids = [row[0] for row in db.cursor.fetchall()]

deliveries_data = [
    [

        fake.uuid4(),
        random.choice(order_ids),
        random.choice(delivery_person_ids),
        random.choice(["On the way", "Delivered"]),
        round(random.uniform(1, 20), 1),
        random.randint(20, 60),
        fake.date_time_between(start_date="-2y",end_date="now").strftime("%Y-%m-%d %H:%M:%S"),
        round(random.uniform(20, 150), 2),
        random.choice(["Bike", "Car", "Bicycle"]),
    ]
    for _ in range(500)
]
db.insert_deliveries(deliveries_data)


connection successfully
Database successfully created
Database successfully created
